In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import re

from time import sleep
from tqdm import tqdm
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

In [2]:
SSD = ['SSD6']
DATE = ['20230312', '20230313']
SESSION = ['SE_Lek1']
DRONE = ['P1D1', 'P1D2']

In [3]:
# Function to calculate distance matrix between territories in two frames
def calculate_distance_matrix(frame1, frame2):
    coords_frame1 = df[df['frame'] == frame1][['pos_x', 'pos_y']].values
    coords_frame2 = df[df['frame'] == frame2][['pos_x', 'pos_y']].values
    distance_matrix = cdist(coords_frame1, coords_frame2)
    return distance_matrix, coords_frame1, coords_frame2

In [4]:
# Function to update territory_id based on optimal assignment
def update_territory_id(frame1, frame2):
    distance_matrix, coords_frame1, coords_frame2 = calculate_distance_matrix(frame1, frame2)
    row_ind, col_ind = linear_sum_assignment(distance_matrix)
    
    global new_id_counter  # Access the global counter
    
    # Update existing territories
    for idx1, idx2 in zip(row_ind, col_ind):
        territory_id_frame1 = df.loc[(df['frame'] == frame1)].iloc[idx1]['territory_id']
        df.loc[(df['frame'] == frame2) & (df['pos_x'] == coords_frame2[idx2, 0]) & (df['pos_y'] == coords_frame2[idx2, 1]), 'territory_id'] = territory_id_frame1
    
    # Handle unassigned territories in frame2
    unassigned_territories = set(range(len(coords_frame2))) - set(col_ind)
    for idx2 in unassigned_territories:
        new_territory_id = n_territories + new_id_counter
        new_id_counter += 1
        df.loc[(df['frame'] == frame2) & (df['pos_x'] == coords_frame2[idx2, 0]) & (df['pos_y'] == coords_frame2[idx2, 1]), 'territory_id'] = new_territory_id

In [ ]:
for ssd in SSD:
    for date in DATE:
        for session in SESSION:
            for drone in DRONE:
                internal_path = '/home/vsridhar/DATA/DeepLabCut/projects/06_TerritoryDetectionP1-Vivek-2023-08-01/output/' + ssd + '/' + date + '/' + session + '/' + drone
                video_name = date + '_' + session + '_' + drone
                
                predictions = glob.glob(internal_path + '/' + video_name + '*full.pickle')
                for pred in predictions:
                    data = pd.read_pickle(pred)
                    frames = list(data.keys())[1:]

                    for idx,frame in enumerate(frames):
                        fr = np.repeat(int(re.findall(r'\d+', frame)[0]),len(data[frame]['coordinates'][0][0]))
                        pos_x = data[frame]['coordinates'][0][0][:,0]
                        pos_y = data[frame]['coordinates'][0][0][:,1]
                        territory_id = np.arange(0,len(data[frame]['coordinates'][0][0]))

                        tmp = pd.DataFrame(np.array([fr,pos_x,pos_y,territory_id]).T, columns=['frame', 'pos_x', 'pos_y', 'territory_id'])
                        if idx != 0:
                            df = pd.concat((df, tmp), axis=0)
                        else:
                            df = tmp

                    # Total number of territories and counter to track new territory IDs
                    n_territories = np.max(df['territory_id'])
                    new_id_counter = 1
                    
                    # Update territory_id for successive frames in reverse order
                    unique_frames = df['frame'].unique()[::-1]
                    for current_frame, previous_frame in tqdm(zip(unique_frames[:-1], unique_frames[1:])):
                        update_territory_id(current_frame, previous_frame)
                        sleep(0.001)
                        
                    df.to_csv(internal_path + '/' + video_name + '_DJI_' + pred[141:145] + '.csv')

5813it [04:51, 19.92it/s]
5813it [05:10, 18.75it/s]
5815it [04:56, 19.59it/s]
5813it [05:11, 18.66it/s]
5814it [04:39, 20.79it/s]
5805it [05:29, 17.59it/s]
5804it [05:26, 17.75it/s]
5814it [05:09, 18.81it/s]
5812it [05:12, 18.61it/s]
5814it [04:51, 19.97it/s]
5811it [04:51, 19.92it/s]
5806it [05:16, 18.34it/s]
5815it [04:53, 19.82it/s]
1it [00:00, 39.75it/s]
5813it [04:47, 20.22it/s]
5814it [04:42, 20.61it/s]
212it [00:05, 37.58it/s]
5803it [04:43, 20.46it/s]
5810it [04:42, 20.60it/s]
5813it [05:01, 19.30it/s]
5000it [03:35, 23.16it/s]
5647it [04:49, 19.47it/s]
5813it [04:45, 20.39it/s]
5814it [04:44, 20.46it/s]
5814it [04:35, 21.08it/s]
5813it [04:34, 21.16it/s]
1657it [01:02, 26.67it/s]
5813it [05:03, 19.17it/s]
5803it [05:29, 17.59it/s]
5803it [06:14, 15.49it/s]
3885it [02:49, 22.86it/s]
5813it [05:27, 17.73it/s]
5814it [05:34, 17.36it/s]
5634it [04:40, 20.07it/s]
5814it [05:35, 17.35it/s]
5816it [04:47, 20.22it/s]
5814it [05:38, 17.18it/s]
5804it [05:33, 17.41it/s]
5816it [04:45, 2